<img src=images/xd-logo.png align=right width=300px>

# FastAPI
*In the begining there was darkness (django and flask), now we have FastAPI*.

- What is a (RESTful) API?
- Introduction to FastAPI

You can access the official FastAPI documentation [here](https://fastapi.tiangolo.com/).

## What's an API?

- An API (application programming interface) is a very generic concept. It is the set of rules that define how to communicate with a software application. For example:
  - The public elements of a software library are "the API" of the library.
    - e.g. Pandas `DataFrame`, the methods the class implements, etc. conform the "Pandas API".
  - Visiting any website is essentially querying a webserver's API that returns HTML.
  - An application can expose an API online that you can use to interact programmatically with it.
    - e.g. `https://www.google.com/search?q={{QUERY}}` is Google's search engine API.

## What is a RESTful API?

[REST](https://en.wikipedia.org/wiki/REST) stands for Representational State Transfer, and it is a set of **standards** that specify how an API should behave. 
- Uniformity across APIs.
- Aiming for reliable and performance communication.
- Designed originally to specify how  different applications shoul communicate over the internet.

At the most basic level an API is a program in a server that processes **requests** from clients, does some work, and returns **responses**.

There are different types of methods that you can pass to a request.

- `GET` Retrieve the resource from the server, **should not mutate** the state of the server (except logs, etc.)
- `POST` Create a resource on the server.
- `PUT` Update the resource on the server.
- `DELETE` Delete the resource from the server.
- ... Some more exotic ones.

There are [many different response types](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) and they're typically clustered together in a number. The most common responde codes are:

- `2xx` response sent.
- `4xx` the server thinks the client made an error.
- `5xx` the server thinks that it made an error.

### What, why, FastAPI?

- Django and Flask are older, bigger, more complex projects. They focus on building webapps with APIs, not just on creating APIs.
- FastAPI is exclusively focussed on creating APIS.
- FastAPI is extremelly fast and supports concurrent operations (more on that later) by default. It is built on top of [Starlette](https://www.starlette.io/).
- Follows OpenAPI standards (e.g. standard JSON schemas that communicate how the API behaves, rules for code-generation).
- Automatic data-model validation (built on top of [Pydantic](https://docs.pydantic.dev/latest/)) and documentation.
- Type-safe (at least relatively safe for python standards).
- Built-in Swagger UI (most popular API tooling framework, original creators of OpenAPI).

## FastAPI basics

FastAPI takes care of all the API machinery (e.g. processing requests, validating input, generating responses), but it needs to be served by a webserver. 

In production settings the most feature-complete, performant, robust and popular server is *nginx*. However, *uvicorn* is a more minimal and also popular alternative that's easier to get started with in a Python environment.

Create a file called `hello_api.py` with the code from the cell below and start a server serving the API with `uvicorn hello_api:app --reload`. By default the server is exposed in port 8000, so the API is reachable at: http://127.0.0.1:8000/ .


The main syntaxt of the uvicorn CLI is `uvicorn FILE:OBJECT`, and `--reload` specifies that the server will atomatically detect changes when the underlying python files change.

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def hello():
    return "Hello, world!"

Have a look at the standard output of the uvicorn CLI to see which requests the server receives and which responses it sends out. What happens if you try to request a different URL e.g. http://127.0.0.1:8000/wrongurl ?

In APIs terminology we call the main access point the "entrypoint" (e.g. port 8000 of the localhost) and each specific URL "endpoints".

You can use the `/docs` endpoint (i.e. http://127.0.0.1:8000/docs) to access the built-in documentation, where you can see all endpoints and interact with them.

## API Parameters

Let's add a new endpoint by adding the following code to `hello_api.py` and try calling the endpoint with different values of `name`.

In [ ]:
@app.get("/greet/{name}")
def greet(name: str):
    return f"Hello, {name}!"

These kind of endpoints accept **"path parameters"**, since the endpoint accepts arguments as part of it's URL path.

If you define an entrypoint with a function that accepts arguments that are not path parameters, FastAPI will assume they are **"query parameters"**. You can pass query parameters to an API after a `?`, and separate multiple ones with `&`. 

E.g. `entrypoint/endpoint?arg_1=val_1&arg_2=val_2` .

Try adding the code in the next code cell to `hello_api.py` and explore:

- What happens if you access http://127.0.0.1:8000/greet_int?num=3
- What happens if you don't provide an argument to the query.
  - What happens if you provide an non-keyword argument? i.e. http://127.0.0.1:8000/greet_int?3
- What happens if you provide extra query arguments?
- Can you define default parameter values?
  - Tip: You can mark arguments as optional without providing a default value by setting their type as `arg: type | None = None`.
- What happens if you pass a value that is not an `int`? 
  - *(Spoilers: FastAPI performs automatic data validation and type casting thanks to Pydantic!)*
- Try extending the endpoint definition to accept both path parameters and query parameters simultaneously.

In [ ]:
@app.get("/greet_int")
def greet_int(num: int):
    return f"Hello, number {num}!"

There's and additional common kind of parameters that FastAPI supports called **"request bodies"**, but to use those you first need to get familiar with the data-validation library that FastAPI builds from: Pydantic.

As a summary of how to specify the type API parameters:

- If the parameter is also **declared in the path**, it will be used as a path parameter.
- If the parameter is of a **simple**-ish **type** (e.g. `int`, `float`, `str`, `bool`) it will be interpreted as a query parameter.
- If the parameter is a **Pydantic model**, it will be interpreted as a request body.


### Mislacenea tips

> If you need to access the raw request for some reason (e.g. to use information about the *requester*), you can define a paramter in your endpoint definitions of type `fastapi.Request` and FastAPI will pass the request as that argument.

> If you like FastAPI's API and need to write CLI applications, take a look at [Typer](https://typer.tiangolo.com/).